In [1]:
# !pip install pytorch-pretrained-bert

In [1]:
from pytorch_pretrained_bert import BertModel, BertTokenizer
import numpy as np
import torch

In [5]:
# 加载bert的分词器
tokenizer = BertTokenizer.from_pretrained('D:/Develop/workspace/datasets/ppb/bert-base-uncased-vocab.txt')
# 加载bert模型，这个路径文件夹下有bert_config.json配置文件和model.bin模型权重文件
bert = BertModel.from_pretrained('D:/Develop/workspace/datasets/ppb/bert-base-uncased/bert-base-uncased/')

In [6]:
s = "I'm not sure, this can work, lol -.-"

tokens = tokenizer.tokenize(s)
print("\\".join(tokens))

i\'\m\not\sure\,\this\can\work\,\lo\##l\-\.\-


In [7]:
# "i\\'\\m\\not\\sure\\,\\this\\can\\work\\,\\lo\\##l\\-\\.\\-"
# 是否需要这样做？
# tokens = ["[CLS]"] + tokens + ["[SEP]"]

ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)])
print(ids.shape)

torch.Size([1, 15])


In [8]:
result = bert(ids, output_all_encoded_layers=True)
print(result)

([tensor([[[ 0.0149,  0.2906, -0.3904,  ...,  0.2831,  0.2524,  0.0099],
         [-0.3020,  0.0948, -0.3686,  ...,  0.9025,  0.6680, -1.3179],
         [ 0.2868,  0.6225,  0.0466,  ..., -0.2463,  0.6610, -0.7523],
         ...,
         [ 0.8117, -0.0816,  0.1211,  ...,  0.0701,  1.2127, -0.0546],
         [ 0.1606,  0.3047, -0.5439,  ..., -0.4230,  0.2595,  0.4675],
         [ 0.6798,  0.0356, -0.3259,  ...,  0.8304,  0.4200,  0.8369]]],
       grad_fn=<AddBackward0>), tensor([[[ 0.1213, -0.1440, -0.3330,  ...,  0.3816,  0.3902, -0.3032],
         [-0.4343,  0.2968,  0.0816,  ...,  1.3178,  1.1097, -1.8229],
         [ 0.1688,  0.7390,  0.2408,  ...,  0.1749,  0.4786, -0.2924],
         ...,
         [ 1.0885,  0.3901,  0.9672,  ...,  0.6685,  1.9166, -0.2629],
         [ 0.1387,  0.0637, -0.3391,  ..., -0.2014,  0.3827,  0.6439],
         [ 0.6293,  0.9271,  0.5554,  ...,  1.2325,  0.2373,  0.6565]]],
       grad_fn=<AddBackward0>), tensor([[[ 0.1505, -0.4199, -0.2121,  ...,  0.4199

In [9]:
from torch import  nn

In [10]:
class CustomModel(nn.Module):

    def __init__(self, bert_path, n_other_features, n_hidden):
        super().__init__()
        # 加载并冻结bert模型参数
        self.bert = BertModel.from_pretrained(bert_path)
        for param in self.bert.parameters():
            param.requires_grad = False
        self.output = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(768 + n_other_features, n_hidden),
            nn.ReLU(),
            nn.Linear(n_hidden, 1)
        )
    def forward(self, seqs, features):
        _, pooled = self.bert(seqs, output_all_encoded_layers=False)
        concat = torch.cat([pooled, features], dim=1)
        logits = self.output(concat)
        return logits

In [12]:
s = "I'm not sure, this can work, lol -.-"

tokens = tokenizer.tokenize(s)
ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)])

model = CustomModel('D:/Develop/workspace/datasets/ppb/bert-base-uncased/bert-base-uncased/',10, 512)
outputs = model(ids, torch.rand(1, 10))
print(outputs)

tensor([[0.0491]], grad_fn=<AddmmBackward0>)
